In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import ariel_load
import copy
import tqdm
kgs.sanity_checks_active = True
kgs.sanity_checks_without_errors = False

local


In [2]:
train_data = kgs.load_all_train_data()
test_data = kgs.load_all_test_data()
len(train_data), len(test_data)

(1100, 1)

In [ ]:
loaders = ariel_load.default_loaders()
for d in tqdm.tqdm(train_data):
    dd = copy.deepcopy(d)
    for t in dd.transits:
        t.load_to_step(2,dd,loaders)
    kgs.print_sanity_checks()

  0%|                                                                                          | 0/1100 [00:00<?, ?it/s]

0.1
enable
-np.inf
enable
0.1


  0%|                                                                                  | 1/1100 [00:02<38:51,  2.12s/it]

enable
-np.inf
enable
gain [0.4369, 0.4369] [0.4, 0.5] 1
offset [-1000.0, -1000.0] [-1001, -999] 2
ratio_hot [0.007203014184397163, 0.0107421875] [0, 0.1] 3
ratio_dead [0.0013297872340425532, 0.0029296875] [0, 0.005] 4
linear_corr_impact [5241.4000882605615, 5511.118858177622] [0, 50000] 9
flat_min [0.888528453936993, 0.9478742031905644] [-inf, 1.1] 7
flat_max [1.0513527685506165, 1.0837272656512849] [0.9, 1.2] 8
0.1
enable
-np.inf
enable
0.1
enable
-np.inf
enable
0.1
enable
-np.inf
enable


  0%|▏                                                                                 | 2/1100 [00:05<47:19,  2.59s/it]

...

0.1
enable
-np.inf
enable
gain [0.4369, 0.4369] [0.4, 0.5] 1
offset [-1000.0, -1000.0] [-1001, -999] 2
ratio_hot [0.0, 0.0107421875] [0, 0.1] 3
ratio_dead [0.0013297872340425532, 0.0029296875] [0, 0.005] 4
linear_corr_impact [4909.047606645341, 6220.785596254689] [0, 50000] 9
flat_min [0.8617695468230018, 0.9478742031905644] [-inf, 1.1] 7
flat_max [1.0383349568133693, 1.0837272656512849] [0.9, 1.2] 8
0.1
enable
-np.inf
enable


  1%|▊                                                                                | 11/1100 [00:17<25:49,  1.42s/it]

0.1
enable
-np.inf
enable
gain [0.4369, 0.4369] [0.4, 0.5] 1
offset [-1000.0, -1000.0] [-1001, -999] 2
ratio_hot [0.0, 0.0107421875] [0, 0.1] 3
ratio_dead [0.0013297872340425532, 0.0029296875] [0, 0.005] 4
linear_corr_impact [4909.047606645341, 6270.582614592728] [0, 50000] 9
flat_min [0.8617695468230018, 0.9478742031905644] [-inf, 1.1] 7
flat_max [1.0383349568133693, 1.0837272656512849] [0.9, 1.2] 8
0.1
enable
-np.inf
enable


  1%|▉                                                                                | 12/1100 [00:19<25:52,  1.43s/it]

0.1
enable
-np.inf
enable
gain [0.4369, 0.4369] [0.4, 0.5] 1
offset [-1000.0, -1000.0] [-1001, -999] 2
ratio_hot [0.0, 0.0107421875] [0, 0.1] 3
ratio_dead [0.0013297872340425532, 0.0029296875] [0, 0.005] 4
linear_corr_impact [4909.047606645341, 6270.582614592728] [0, 50000] 9
flat_min [0.8617695468230018, 0.9478742031905644] [-inf, 1.1] 7
flat_max [1.0383349568133693, 1.0837272656512849] [0.9, 1.2] 8


In [4]:
kgs.print_sanity_checks()

gain [0.4369, 0.4369] [0.4, 0.5] 1
offset [-1000.0, -1000.0] [-1001, -999] 2
ratio_hot [0.0, 0.0107421875] [0, 0.015] 3
ratio_dead [0.0013297872340425532, 0.0029296875] [0, 0.005] 4
linear_corr_impact [3947.419761334895, 33993.462981898854] [0, 50000] 9
dark_min [-0.0007936957993979564, 0.002066195137730647] [-0.0015, 0.004] 5
dark_max [0.01163899428117778, 0.013298721036968861] [0.0, 0.02] 6
flat_min [0.8617695468230018, 0.9478742031905644] [0.7, 1.1] 7
flat_max [1.0383349568133693, 1.0837272656512849] [0.9, 1.2] 8
